In [39]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [40]:
# Defining paths
train_dir = r'C:\Users\Kruti Agrawal\Desktop\Projects\ASL_detection\ASL_detection\asl_alphabet_train\asl_alphabet_train'

In [41]:
# Loading data
def load_data(train_dir):
    images = []
    labels = []
    class_labels = os.listdir(train_dir)
    class_labels.sort()  

    for label in class_labels:
        class_path = os.path.join(train_dir, label)
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            image = cv2.imread(image_path)
            images.append(image)
            labels.append(class_labels.index(label))  

    return np.array(images), np.array(labels), class_labels

images, labels, class_labels = load_data(train_dir)

In [42]:
images = images.astype('float32') / 255.0

MemoryError: Unable to allocate 38.9 GiB for an array with shape (87000, 200, 200, 3) and data type float32

In [ ]:
# Converting labels to categorical
labels = to_categorical(labels, num_classes=len(class_labels))

In [ ]:
# Splitting data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Creating an image data generator with augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Fitting the generator on training data
datagen.fit(X_train)

In [ ]:
# Loadong MobileNetV2 as the base model
base_model = MobileNetV2(input_shape=(64, 64, 3), include_top=False, weights='imagenet')
base_model.trainable = False  

In [ ]:
# Adding custom layers on top of MobileNetV2
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),  # Additional dense layer
    Dropout(0.3),
    Dense(len(class_labels), activation='softmax')
])

In [ ]:
#for layer in base_model.layers[-20:]: 
    #layer.trainable = True
    
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Defining a learning rate reducer
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

In [ ]:
# Training the model with the data generator and callbacks
model.fit(datagen.flow(X_train, y_train, batch_size=32), 
          validation_data=(X_val, y_val), 
          epochs=50,  
          callbacks=[lr_reducer])

In [ ]:
# Predictions on validation data
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
# Classification report
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

In [ ]:
# Saving the trained model for later use in Streamlit
model.save('asl_model.h5')